In [ ]:
'''
This notebook is for exploratory data analysis of the raw AIS data
'''

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import warnings
import sys
#warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))

In [ ]:
# add paths for modules
sys.path.append('../src/visualization')
sys.path.append('../src/features')
print(sys.path)

# import modules
import visualize

In [ ]:
# read data from file
filename = '../data/raw/AIS_04-09_2022_stavanger/ais_202204.csv'
df = pd.read_csv(filename, delimiter=';', decimal='.')
df.info(show_counts=True)

In [ ]:
# convert to geopandas df
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
df = []  # free memory
n_messages = len(gdf)

In [ ]:
# find duplicates
duplicates = gdf.duplicated(subset=['mmsi', 'lat', 'lon'], keep='first')
n_duplicates = len(gdf[duplicates])

In [ ]:
# drop duplicate AIS data (reported by multiple stations)
cleaned_gdf = gdf.drop_duplicates(subset = ['mmsi', 'lat', 'lon'],
                                    keep = 'first')
print(f'Dropped {n_duplicates} duplicates ({n_duplicates/n_messages*100:.2f}%)')

In [ ]:
gdf.info()  # print information about loaded data

In [ ]:
gdf.head()

In [ ]:
# some dataset characteristics
columns = ['mmsi', 'imo_nr', 'nav_status', 'message_nr']
for column in columns:
    print(f'{gdf[column].nunique()} unique {column}')

# print nav_status and message_nr values
print('\n')
print(f'Unique nav_status: \n {sorted(gdf.nav_status.unique())}')
print(f'Unique message_nr: \n {sorted(gdf.message_nr.unique())}')
msg_nr_counts = gdf['message_nr'].value_counts()
msg_nr_distribution = msg_nr_counts / len(gdf)
msg_nr_distribution = msg_nr_distribution.sort_index()
nav_status_counts = gdf['nav_status'].value_counts()
nav_status_distribution = nav_status_counts / len(gdf)
nav_status_distribution = nav_status_distribution.sort_index()

In [ ]:
# join ship metadata to get ship category distribution
sys.path.append('../src/datawrangling')
from make_trajectories_from_AIS import add_ship_metadata
metadata_filename = '../data/external/seilas-2022.csv'
gdf = add_ship_metadata(metadata_filename, gdf)

In [ ]:
# print unique ship categories
gdf.skipsgruppe.unique()

In [ ]:
# Translate ship categories to English
gdf['skipsgruppe'] = gdf['skipsgruppe'].replace(to_replace=['Last', 'Fisk', 'Passasjer', 'Slep'], 
                                                value=['Cargo', 'Fishing', 'Passenger', 'Tug'])
gdf['skipsgruppe'].fillna('Unknown', inplace=True)
gdf.rename(columns={'skipsgruppe': 'ship_category'}, inplace=True)

In [ ]:
# plot distribution of nav_status and message_nr and ship category

# distribution of message_nr
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
msg_nr_distribution.plot(kind='bar', ax=axes[0])
axes[0].set_title('Distribution of message_nr')
axes[0].set_ylabel('fraction of AIS messages')

# distribution of nav_status
nav_status_distribution.plot(kind='bar', ax=axes[1])
axes[1].set_title('Distribution of nav_status')

# distribution of ship_category
category_counts = gdf['ship_category'].value_counts()
category_distribution = category_counts / len(gdf)
category_distribution.sort_values(inplace=True, ascending=False)
category_distribution.plot(kind='bar', ax=axes[2])
axes[2].set_title('Distribution of ship_category')
fig.subplots_adjust(bottom=0.3)

plt.savefig('distribution of message_nr, nav_status and ship_category.pdf')
plt.show()

In [ ]:
# investigate duplicates
mmsi = 257088660
one_ship_gdf = gdf[gdf.mmsi == mmsi]

duplicates = one_ship_gdf.duplicated(subset=['mmsi', 'lat', 'lon'], keep='first')
n_duplicates = len(one_ship_gdf[duplicates])
print(len(one_ship_gdf), n_duplicates)
print(duplicates)

In [ ]:
# plot data (n points) to visualize duplicates
n_points = 1000  # -1 selects all points
if n_points > 0:
    # selection = np.random.randint(0, high=len(gdf), size=n_points)
    #map = gdf.iloc[0:n_points].explore(cmap='jet', column='mmsi', name='AIS messages')
    map = one_ship_gdf.iloc[0:n_points].explore(color='blue', name='AIS messages')
    map = one_ship_gdf[duplicates].iloc[0:n_points].explore(m=map, color='red', name='duplicates')
else:
    map = gdf.explore(cmap='jet', column='mmsi', name='AIS messages')

In [ ]:
# get a bounding box of the geographic area
bbox = visualize.get_bounding_box(gdf)
map = bbox.explore(m=map, color='red', name='bounds')
folium.LayerControl().add_to(map)
map

In [ ]:
# make hexbin plots of different ship types
ship_types = ['Cargo', 'Fishing', 'Passenger', 'Auxiliary']
#ship_types = ['Passenger', 'Fishing']
gdf['ship_category'] = gdf['ship_category'].replace(to_replace=['Tug'], 
                                                value=['Auxiliary'])
map = folium.Map(location=[59, 5], tiles="OpenStreetMap", zoom_start=8)
for ship_type in ship_types:
    sub_gdf = gdf[gdf.ship_category == ship_type]
    map=visualize.traffic_raster_overlay(sub_gdf, map, overlay_name=ship_type)
folium.LayerControl().add_to(map)
map.save('../reports/maps/raw_AIS_by_ship_category.html')
map

In [ ]:
# map of stop points
map = folium.Map(location=[59, 5], tiles="OpenStreetMap", zoom_start=8)
sub_gdf = gdf[gdf.nav_status.isin([1, 5])]
map=visualize.traffic_raster_overlay(sub_gdf, map, overlay_name='ports and anchor places')
folium.LayerControl().add_to(map)
map.save('../reports/maps/stop_points.html')
map

In [ ]:
# convert raw AIS data to trajectory collection
import movingpandas as mpd
trajectories = mpd.TrajectoryCollection(gdf, traj_id_col='mmsi', 
                                        obj_id_col='mmsi', t='date_time_utc')

In [ ]:
# visualize a single trajectory
# We want to show here, that splitting a single trajectory into sub-trajectories is necessary
from datetime import timedelta, datetime
warnings.filterwarnings('ignore')
mmsis = gdf.mmsi.unique()
n=3
selection = mmsis[n:n+1]
selected_trajectories = trajectories.filter('mmsi', selection)
obs_split_trajectories = mpd.ObservationGapSplitter(selected_trajectories).split(gap=timedelta(minutes=10), min_length=500)
print(f'Observation Gap splitter split {len(trajectories)} trajectories into {len(obs_split_trajectories)} sub-trajectories')

# Split trajectories into sub trajectories when a stop longer than 30 seconds is observed (for example ferries). Only keep trajectories longer than 500m.
split_trajectories = mpd.StopSplitter(obs_split_trajectories).split(max_diameter=50, min_duration=timedelta(minutes=0.5), min_length=500)
print(f'Stop splitter split {len(obs_split_trajectories)} trajectories into {len(split_trajectories)} sub-trajectories')

map = split_trajectories.to_traj_gdf()[['geometry', 'mmsi']].explore(cmap='jet', column='mmsi', name='split trajectories', legend=False)
map = selected_trajectories.to_traj_gdf()[['geometry', 'mmsi']].explore(m=map, name='raw trajectories')
folium.LayerControl().add_to(map)
map

In [ ]:
map.save('../reports/maps/raw_trajectories.html')